In [1]:
# 查看样本信息
import pandas as pd
import re

train_df=pd.read_csv('data/data130186/train_public.csv')
test_df=pd.read_csv('data/data130187/test_public.csv')

# 固定随机种子，使结果可复现
import random
import numpy as np
import paddle  #百度提出的深度学习框架

seed = 1024
random.seed(seed)   # Python random module.
np.random.seed(seed) # Numpy module.
paddle.seed(seed)   # paddle module.

In [2]:
# 查看空值分布情况
train_df.isnull().any()

loan_id                     False
user_id                     False
total_loan                  False
year_of_loan                False
interest                    False
monthly_payment             False
class                       False
employer_type               False
industry                    False
work_year                    True
house_exist                 False
censor_status               False
issue_date                  False
use                         False
post_code                   False
region                      False
debt_loan_ratio             False
del_in_18month              False
scoring_low                 False
scoring_high                False
known_outstanding_loan      False
known_dero                  False
pub_dero_bankrup             True
recircle_b                  False
recircle_u                  False
initial_list_status         False
app_type                    False
earlies_credit_mon          False
title                       False
policy_code   

In [3]:
# 新特征构造
train_df['risk'] = train_df['total_loan'] * train_df['interest']  #贷款风险=贷款数额*当前贷款利率
test_df['risk'] = test_df['total_loan'] * test_df['interest']

# class数量化
class_dic = dict(zip([chr(y) for y in range(ord('A'), ord('G') + 1)], range(0, 8)))  #把网络贷款登记A-G数量化0-7
train_df['class'] = train_df['class'].apply(lambda x: class_dic[x])
test_df['class'] = test_df['class'].apply(lambda x: class_dic[x])

# region重新构造，分成两派即可
region_des = train_df.groupby(['region'])['isDefault'].describe()
default_regions = region_des[region_des['mean'] > 0.2].index.tolist()
train_df['region'] = train_df['region'].apply(lambda x: 1 if x in default_regions else 0)  #1代表在默认的区域，0代表不在
test_df['region'] = test_df['region'].apply(lambda x: 1 if x in default_regions else 0)

# work_year数量化
work_year_dict = {
    '< 1 year': 0,
    '1 year': 1,
    '2 years': 2,
    '3 years': 3,
    '4 years': 4,
    '5 years': 5,
    '6 years': 6,
    '7 years': 7,
    '8 years': 8,
    '9 years': 9,
    '10+ years': 10,
}

train_df['work_year'] = train_df['work_year'].map(work_year_dict) #map 是python的映射函数
test_df['work_year'] = test_df['work_year'].map(work_year_dict)

# earlies_credit_mon处理

month_dict = {'Dec': 12,
         'Apr': 4,
         'Oct': 10,
         'Jun': 6,
         'May': 5,
         'Jan': 1,
         'Nov': 11,
         'Jul': 7,
         'Aug': 8,
         'Feb': 2,
         'Sep': 9,
         'Mar': 3}

train_df['early_mon'] = train_df['earlies_credit_mon'].apply(lambda x: month_dict[re.findall('[a-zA-Z]+', x)[0]])
test_df['early_mon'] = test_df['earlies_credit_mon'].apply(lambda x: month_dict[re.findall('[a-zA-Z]+', x)[0]])

train_df['early_day'] = train_df['earlies_credit_mon'].apply(lambda x: eval(re.findall('[0-9]+', x)[0]))
test_df['early_day'] = test_df['earlies_credit_mon'].apply(lambda x: eval(re.findall('[0-9]+', x)[0]))

# f系列新构造一个特征
train_df['f0xf3'] = train_df['f0'] * train_df['f3']
test_df['f0xf3'] = test_df['f0'] * test_df['f3']

# 缺失值均值填充（取平均值）

train_df['pub_dero_bankrup'] = train_df['pub_dero_bankrup'].fillna(train_df['pub_dero_bankrup'].mean())
test_df['pub_dero_bankrup'] = test_df['pub_dero_bankrup'].fillna(test_df['pub_dero_bankrup'].mean())

'''
以下是不成功的特征构造：

# f系列特征重新构造
f_ls = ['f0', 'f1', 'f2', 'f3', 'f4']

for i in f_ls:
    for j in f_ls[f_ls.index(i)+1:]:
        train_df['%sx%s'%(i,j)] = train_df[i] * train_df[j]
        test_df['%sx%s'%(i,j)] = test_df[i] * test_df[j]

# use重新构造，分成两派即可
use_des = train_df.groupby(['use'])['isDefault'].describe()
default_uses = use_des[use_des['mean'] > 0.2].index.tolist()
train_df['use'] = train_df['use'].apply(lambda x: 1 if x in default_uses else 0)
test_df['use'] = test_df['use'].apply(lambda x: 1 if x in default_uses else 0)

# post_code重新构造，分成两派即可
post_code_des = train_df.groupby(['post_code'])['isDefault'].describe()
default_post_codes = post_code_des[post_code_des['mean'] > 0.2].index.tolist()
train_df['post_code'] = train_df['post_code'].apply(lambda x: 1 if x in default_post_codes else 0)
test_df['post_code'] = test_df['post_code'].apply(lambda x: 1 if x in default_post_codes else 0)
'''

train_df.to_csv('train_df.csv', index=False)
test_df.to_csv('test_df.csv', index=False)

In [4]:
train_df['isDefault'].value_counts()

0    8317
1    1683
Name: isDefault, dtype: int64

In [5]:
# 数值特征描述
pos = train_df[train_df['isDefault']==1].describe()
pos_des = pd.DataFrame(pos.values.T, index=pos.columns, columns=pos.index)  # 转置操作
pos_des = pos_des['mean']

neg = train_df[train_df['isDefault']==0].describe()
neg_des = pd.DataFrame(neg.values.T, index=neg.columns, columns=neg.index)
neg_des = neg_des['mean']

# 数值特征筛选
des = pd.DataFrame({'pos': pos_des, 'neg': neg_des})
des['diff'] = (des['pos'] - des['neg'])/des['neg']
des = des.sort_values(by=['diff'])
num_item = des[abs(des['diff'])>=0.01].index.tolist()[:-1]    #索引转化为列表

un_num_item = ['early_mon', 'work_year', 'region', 'employer_type','industry', 'house_exist', 'censor_status',
        'use',
        'initial_list_status','app_type',
        'policy_code']

for i in un_num_item:
        try:
                num_item.remove(i)
        except:
                continue

In [6]:
len(num_item)

26

## 构造dataset

In [7]:
import paddle.vision.transforms as T
from PIL import Image

class MyDateset(paddle.io.Dataset):
    # csv_dir对应要读取的数据地址，standard_csv_dir用于生成均值和方差信息对数据进行归一化的文件地址
    def __init__(self,csv_dir,standard_csv_dir='train_df.csv',mode = 'train'):
        super(MyDateset, self).__init__()

        # 读取数据
        self.df = pd.read_csv(csv_dir)
        
        # 构造各个变量的均值和方差
        st_df = pd.read_csv(standard_csv_dir)
        self.mean_df = st_df.mean()
        self.std_df = st_df.std()

        # 分别指定数值型变量/分类变量/不使用的变量
        self.num_item = num_item
        self.un_num_item = un_num_item

        # 构造一个映射表，将分类变量/分类字符串映射到对应数值上
        un_num_item_list = {}
        for item in self.un_num_item:
            un_num_item_list[item]=list(set(st_df[item].values))
        self.un_num_item_list = un_num_item_list

        self.mode = mode

    def __getitem__(self, index):
        data=[]

        # 进行归一化，如果这个数值缺省了直接设置为0
        for item in self.num_item:
            if np.isnan(self.df[item][index]):
                data.append((0-self.mean_df[item])/self.std_df[item])
            else:
                data.append((self.df[item][index]-self.mean_df[item])/self.std_df[item])
        
        emb_data = []

        # 将分类变量映射到对应数值上
        for item in self.un_num_item:
            try:
                if self.df[item][index] not in self.un_num_item_list[item]:
                    emb_data.append(-1)
                else:
                    emb_data.append(self.un_num_item_list[item].index(self.df[item][index]))
            except:
                emb_data.append(-1)

        data = paddle.to_tensor(data).astype('float32')
        emb_data = paddle.to_tensor(emb_data).astype('float32')

        # 如果当前模式不为train，则返回对应的loan_id，用于锁定样本条目
        if self.mode == 'train':
            label = self.df['isDefault'][index]
        else:
            label = self.df['loan_id'][index]

        label = np.array(label).astype('int64')
        return data,emb_data,label

    def __len__(self):
        return len(self.df)

In [8]:
dataset=MyDateset('train_df.csv')
[data,emb_data,label] = dataset[0]
print(data.shape)
print(emb_data.shape)
print(label)

print('\n')
print(num_item)

[26]
[11]
0


['early_return_amount_3mon', 'early_return_amount', 'early_return', 'f1', 'early_day', 'title', 'recircle_b', 'scoring_low', 'scoring_high', 'f4', 'monthly_payment', 'post_code', 'total_loan', 'f3', 'recircle_u', 'year_of_loan', 'known_outstanding_loan', 'del_in_18month', 'f0', 'debt_loan_ratio', 'interest', 'f0xf3', 'risk', 'pub_dero_bankrup', 'known_dero', 'class']


In [9]:
# 查看变量分布情况
des

,pos,neg,diff
early_return_amount_3mon,2.803791e+01,3.973950e+02,-0.929446
early_return_amount,2.054415e+02,2.572250e+03,-0.920132
early_return,1.663696e-01,1.518095e+00,-0.890409
f1,1.303781e-03,1.445846e-03,-0.098258
early_day,4.226144e+01,4.620753e+01,-0.085399
title,1.687819e+03,1.832562e+03,-0.078984
use,1.645870e+00,1.786221e+00,-0.078574
recircle_b,1.550428e+04,1.675956e+04,-0.074900
work_year,5.919503e+00,6.091975e+00,-0.028311
scoring_low,6.548115e+02,6.659983e+02,-0.016797


## 构造网络

对于分类变量使用两层全连接成生成embedding（Embedding 是一个将离散变量转为连续向量表示的一个方式）

In [10]:
#构造网络
class MyNet(paddle.nn.Layer):          #in_features (int) – 线性变换层输入单元的数目。out_features (int) – 线性变换层输出单元的数目。
    def __init__(self):
        super(MyNet,self).__init__()
        self.fc = paddle.nn.Linear(in_features=len(num_item), out_features=512)   #fc全连接层
        self.emb1 = paddle.nn.Linear(in_features=len(un_num_item),out_features=2048)    #emb嵌入层将正整数（下标）转换为具有固定大小的向量
        self.emb2 = paddle.nn.Linear(in_features=2048,out_features=512)
        self.emb3 = paddle.nn.Linear(in_features=1024,out_features=512)
        self.dropout = paddle.nn.Dropout(0.1)
        self.relu = paddle.nn.ReLU()    #relu（激活函数），ReLU函数其实是分段线性函数，把所有的负值都变为0，而正值不变
        self.out = paddle.nn.Linear(in_features=512,out_features=2)

 # 定义网络的前向计算
    def forward(self,data,emb_data):
        x = self.fc(data)
        x = self.dropout(x)

        emb = self.emb1(emb_data)
        emb = self.emb2(emb)

        x = paddle.concat([x,emb],axis=-1)   #沿x轴横向联结
        x = self.relu(x)
        x = self.emb3(x)
        x = self.out(x)
        
        x = paddle.nn.functional.softmax(x)    #缩放
        return x


## 训练

In [11]:
# 构造读取器，可以尝试继续调大batch size，当然batch size越大学习率也要相应增大
train_dataset=MyDateset('train_df.csv')
batch_size = 2048    #我们需要决定在一次训练中，要选取多少样本喂给神经网络，这个要选择的样本个数，就是batch size。
train_dataloader = paddle.io.DataLoader(
    train_dataset,        #从此参数给定数据集中加载数据
    batch_size=batch_size,  #每mini-batch中样本个数
    shuffle=True,       #生成mini-batch索引列表时是否对索引打乱顺序
    drop_last=False)    #是否丢弃因数据集样本数不能被 batch_size 整除而产生的最后一个不完整的mini-batch

In [12]:
# 构造模型
model = MyNet()

# model_dict = paddle.load('model.pdparams')
# model.set_dict(model_dict)

model.train()   #训练阶段打开 train模式

max_epoch=17   #迭代次数
opt = paddle.optimizer.SGD(learning_rate=0.1, parameters=model.parameters())   #SGD该接口实现随机梯度下降算法的优化器
#为网络添加反向计算过程，并根据反向计算所得的梯度，更新parameters中的Parameters，最小化网络损失值loss。l-rate,学习率；para优化参数

# 训练
now_step=0
for epoch in range(max_epoch):
    for step, data in enumerate(train_dataloader):
        now_step+=1

        data,emb_data, label = data
        pre = model(data,emb_data)
        loss = paddle.nn.functional.cross_entropy(pre,label,weight=paddle.to_tensor([0.2,1.2]),reduction='mean')   
        #redu.=mean,weight!=none，返回上一步结果的加权平均值
        #将负样本的权重设为0.2，正样本为1.2
        # loss = paddle.nn.functional.square_error_cost(pre,label.reshape([-1,1]).astype('float32'))
        # loss = paddle.mean(loss)
        loss.backward()    #将损失loss 向输入侧进行反向传播
        opt.step()     #优化器对x的值进行更新
        opt.clear_gradients()   #清除需要优化的参数的梯度。
        if now_step%1==0:
            print("epoch: {}, batch: {}, loss is: {}".format(epoch, step, loss.mean().numpy()))

# 保存模型到model.pdparams
paddle.save(model.state_dict(), 'model.pdparams')  

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/math_op_patch.py:253: UserWarning: The dtype of left and right variables are not the same, left dtype is paddle.float32, but right dtype is paddle.bool, the right dtype will convert to paddle.float32
  format(lhs_dtype, rhs_dtype, lhs_dtype))


epoch: 0, batch: 0, loss is: [0.7309555]
epoch: 0, batch: 1, loss is: [0.73801726]
epoch: 0, batch: 2, loss is: [0.68107724]
epoch: 0, batch: 3, loss is: [0.6758552]
epoch: 0, batch: 4, loss is: [0.6810281]
epoch: 1, batch: 0, loss is: [0.67134434]
epoch: 1, batch: 1, loss is: [0.7081331]
epoch: 1, batch: 2, loss is: [0.6483095]
epoch: 1, batch: 3, loss is: [0.7003139]
epoch: 1, batch: 4, loss is: [0.6935282]
epoch: 2, batch: 0, loss is: [0.61797047]
epoch: 2, batch: 1, loss is: [0.616491]
epoch: 2, batch: 2, loss is: [0.6241134]
epoch: 2, batch: 3, loss is: [0.6439018]
epoch: 2, batch: 4, loss is: [0.6535806]
epoch: 3, batch: 0, loss is: [0.59173006]
epoch: 3, batch: 1, loss is: [0.58895606]
epoch: 3, batch: 2, loss is: [0.58412546]
epoch: 3, batch: 3, loss is: [0.5932163]
epoch: 3, batch: 4, loss is: [0.5793675]
epoch: 4, batch: 0, loss is: [0.58987665]
epoch: 4, batch: 1, loss is: [0.5751129]
epoch: 4, batch: 2, loss is: [0.5662258]
epoch: 4, batch: 3, loss is: [0.5792259]
epoch: 4,

## 预测

In [13]:
# 读取模型和构造读取器
model = MyNet()

model_dict = paddle.load('model.pdparams')
# model_dict = paddle.load('model.pdparams')

model.set_dict(model_dict)

model.eval()   #预测阶段打开 eval 模式，模型中的 dropout 等操作会关掉

test_dataset=MyDateset('test_df.csv',mode = 'test')

test_dataloader = paddle.io.DataLoader(
    test_dataset,     #从此参数给定数据集中加载数据
    batch_size=1,    #步长
    shuffle=False,   #是否对索引打乱顺序
    drop_last=False)   #是否丢弃因数据集样本数不能被 batch_size 整除而产生的最后一个不完整的mini-batch，

In [14]:
# 将结果保存在result.csv中
result = []
for step, data in enumerate(test_dataloader):
    data ,emb_data, loan_id = data
    pre = model(data,emb_data)
    result.append([loan_id.numpy()[0], pre[:,1].numpy()[0]])
    # result.append([loan_id.numpy()[0], np.argmax(pre.numpy())])

pd.DataFrame(result,columns=['id','isDefault']).to_csv('result4.csv',index=None)

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 